In [1]:
addpath ("files")
setenv('GNUTERM','qt')
format long

In [142]:
%%file files/gradiente.m
function  [gfx] = gradiente(f, x, h = 1.e-08)

%{
    Aproximacion del gradiente por diferencias hacia adelante 
    de una funcion  de R^n a R.
    
    Input:
        f   .- apuntador a la función
        x   .- vector columna de dimension n.

    Output:

        gfx .- vector columna de dimension n, 
                 es la aproximacion al gradiente en x.
%}
    
    n = length(x);
    
    fx = f(x);                            
    xt = x;
    
    for i = 1:n
        xt(i) = xt(i) + h;
        fxh = f(xt);
        gfx(i) = (fxh - fx)/h;
        xt(i) = x(i);
    end
end

Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/gradiente.m'.


In [8]:
%%file files/hessiana.m
function [H] = hessiana(f, x, h = 1.e-05)

    n = length(x);
    H = zeros(n);
    fx = f(x);

    for i = 1:n
        x1 = x;
        x1(i) = x1(i) + h;
        f1 = f(x1);
        for j = i:n
            x2 = x;
            x2(j) = x2(j) + h;
            f2 = f(x2);

            x3 = x1;
            x3(j) = x3(j) + h;
            f3 = f(x3);
            
            H(i,j) = (f3 - f2 - f1 + fx)/(h^2);
            H(j,i) = H(i,j);
        end
    end
end

Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/hessiana.m'.


In [12]:
%%file files/metodoblhibrido.m

function [W,x,iter] = metodoblhibrido(f, x, method='newton', tol=1.e-08, 
                                      maxiter = 250, maxjter = 20, c1 = 1.e-04)
%{
    Método de búsqueda de línea con la primer condición de Wolfe
    usando máximo descenso.
    usando los métodos de descenso:
        1. Máximo descenso
        2. Método de Newton
    Input:
        
        fname.- cadena de caracteres con el nombre de la función a minimizar.
        x    .- vector n-dimensional.
    
    Output:
        x    .- vector n-dimensional con la aproximación al mínimo local.
        iter .- contador con el número final de iteraciones externas.
%}
    iter = 0;        
    p = 1; 

    g  = gradiente(f,x)';
    W  = [x]; 

    while ( norm(g) > tol && iter < maxiter)

        if (strcmp(method,'max'))
            p = -g;
        end

        if (strcmp(method,'newton'))
            H = hessiana(f,x);
            H = (H' + H)/2;
            p = - chol(H) \g; 
        end

        %-----------------------------------------
        %            Búsqueda híbrida
        
        alfa_back = 1; alfa_inter =1; 
        jter = 0;   
        
        f_x  = f(x);
        xt   = x + p;        
        f1   = f(xt);
                  
        s  = p'*g;  
        %if (abs(s) < 1.e-12)
        %    disp('No existe suficiente descenso  ')
        %    disp(sprintf('%2.0f  %2.10f',iter, s))
        %    break;
        %end

        while(true)  % búsqueda de línea
                
            %-----------------------------------------
            %           Paso Backtracking
            alfa_back = alfa_back/2;
            f_back  = f(x + alfa_back*p);
            wolfe_back = false;
            
            %-----------------------------------------
            %           Paso Interpolación
            d2 = f1 - f_x - s;
            alfa_inter = -(s)/(2*d2); 
            f_inter =f(x + alfa_inter*p);
            wolfe_inter = false;
            
            %-----------------------------------------
            %    Paso backtracking cumple W1
            if (f_back <= f_x + alfa_back*c1*s)
                wolfe_back = true;
            end
            
            %-----------------------------------------
            %    Paso interpolación cumple W1
            if (f_inter <= f_x+ alfa_inter*c1*s)
                wolfe_inter = true;
            end
            
            if (wolfe_back && wolfe_inter)
                %a = 'las dos'
                %jter
                alfa = max(alfa_back,alfa_inter);
            elseif(wolfe_inter)
                %a = 'inter'
                %jter
                alfa = alfa_inter;
            elseif(wolfe_back)
                %a = 'backtrack'
                %jter
                alfa = alfa_back;
            end
            
            if (jter > maxjter) 
                alfa = 1.e-2;
                break;
            end
            
            if (wolfe_back || wolfe_inter)
                break;
            end
             
            f1   = f(x + alfa_inter*p);
            jter = jter +1;
        end   
        
        if ((norm(alfa*p) < 1.e-3) || (jter > maxjter) )
            alfa = 1.e-2;
        end
        
        %          Fin de búsqueda híbrida
        %------------------------------------------
          
        W = [W, x];
        x = x + alfa*p;  
        fx = f(x); 
        g = gradiente(f,x)';
        iter = iter + 1;
        
    end      
end

Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/metodoblhibrido.m'.


In [162]:
%%file files/plot_surface.m
function [] = plot_surface(f, x_dom=[-1,1], y_dom=[-1,1], n=40)
    x = linspace(x_dom(1),x_dom(2),n);
    y = linspace(y_dom(1),y_dom(2),n);
    [X,Y] = meshgrid(x,y);
    surf(X,Y,f(cat(3,X,Y)),'FaceAlpha',0.4)
end

Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/plot_surface.m'.


In [145]:
%%file files/shubert.m
function [ans] = shubert(X)
    if (length(size(X)) > 2)
        x = X(:,:,1);
        y = X(:,:,2);
    else
        x = X(1);
        y = X(2);
    end

    n = length(x);
    sum1 = zeros(1,n);
    sum2 = zeros(1,n);
    
    for i = 1:5
        sum1 = sum1 + i .* cos((i+1).*x+i);
        sum2 = sum2 + i .* cos((i+1).*y+i);
    end

    ans = sum1 .* sum2;
end

Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/shubert.m'.


In [156]:
%%file files/rosenbrock.m
function [fx] = rosenbrock(X)
%{
    Función de Rosenbrock: f: R^2 --> R cuyo mínimo local es muy 
    difícil de alcanzar por medio de los métodos de optimización.
    
    Input:
         x.- vector de longitud 2
    Output:
        fx.- número real con el valor de la función.
%}

    if (length(size(X)) > 2)
        x = X(:,:,1);
        y = X(:,:,2);
    else
        x = X(1);
        y = X(2);
    end
    
    fx = 100.*(y - x.^2).^2 + (1 - x).^2;
end

Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/rosenbrock.m'.


$f(x) = 10f + \sum_{i=1}^d [x_i^2 - 10 \cos(2\pi x_i)]$

In [157]:
%%file files/rastrigin.m
function [fx] = rastrigin(X)

if (length(size(X)) > 2)
    x = X(:,:,1);
    y = X(:,:,2);
else
    x = X(1);
    y = X(2);
end

s = (x.^2 - 10.*cos(2.*pi.*x));
s = s + (y.^2 - 10.*cos(2.*pi.*y));

fx = 10*2 + s;

end


Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/rastrigin.m'.


In [148]:
%%file files/griewank.m

function [y] = griewank(xx)

d = length(xx);
sum_ = 0;
prod_ = 1;

for ii = 1:d
    xi = xx(ii);
    sum_ = sum_ + xi^2/4000;
    prod_ = prod_ * cos(xi/sqrt(ii));
end

y = sum_ - prod_ + 1;

end


Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/griewank.m'.


https://www.sfu.ca/~ssurjano/ackley.html

In [149]:
%%file files/ackley.m
function [y] = ackley(xx, a=20, b=.2, c=2*pi)

d = length(xx);

if (nargin < 4)
    c = 2*pi;
end
if (nargin < 3)
    b = 0.2;
end
if (nargin < 2)
    a = 20;
end

sum1 = 0;
sum2 = 0;
for ii = 1:d
    xi = xx(ii);
    sum1 = sum1 + xi^2;
    sum2 = sum2 + cos(c*xi);
end

term1 = -a * exp(-b*sqrt(sum1/d));
term2 = -exp(sum2/d);

y = term1 + term2 + a + exp(1);

end


Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/ackley.m'.


https://www.sfu.ca/~ssurjano/branin.html

In [150]:
%%file files/branin.m

function [y] = branin(xx, a=1, b=5.1/(4*pi^2), c=5/(pi), r=6, s=10, t=1/(8*pi))
x1 = xx(1);
x2 = xx(2);

if (nargin < 7)
    t = 1 / (8*pi);
end
if (nargin < 6)
    s = 10;
end
if (nargin < 5)
    r = 6;
end
if (nargin < 4)
    c = 5/pi;
end
if (nargin < 3)
    b = 5.1 / (4*pi^2);
end
if (nargin < 2)
    a = 1;
end

term1 = a * (x2 - b*x1^2 + c*x1 - r)^2;
term2 = s*(1-t)*cos(x1);

y = term1 + term2 + s;

end


Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/branin.m'.


https://www.sfu.ca/~ssurjano/easom.html

In [151]:
%%file files/easom.m

function [y] = easom(xx)

x1 = xx(1);
x2 = xx(2);

fact1 = -cos(x1)*cos(x2);
fact2 = exp(-(x1-pi)^2-(x2-pi)^2);

y = fact1*fact2;

end


Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Proyectos/proyecto_1/files/easom.m'.


In [129]:
[W,x,iter] = metodoblhibrido(@rosenbrock,[2,2]','newton',false);
x
rosenbrock(x)

x =

   9.999976694386812e-01
   9.999951296611845e-01

ans =    9.808884243547103e-12


In [130]:
[W,x,iter] = metodoblhibrido(@rastr,[.4,.3]','max',false);
x
rastr(x)

x =

  -1.989141575595430
  -1.990607460801713

ans =  7.959875669344363


In [131]:
[W,x,iter] = metodoblhibrido(@griewank,[2,0]','newton',false);
x
griewank(x)

x =

   3.070148487026073e+00
   1.272486639708042e-03

ans =  1.999805000477428


In [132]:
[W,x,iter] = metodoblhibrido(@ackley,[0,1.5]','max',false);
x
ackley(x)

x =

  -2.257506681591082e-06
  -2.803696601529326e-02

ans =    1.000968658023713e-01


In [133]:
[W,x,iter] = metodoblhibrido(@branin,[-4,13]','newton',false);
x
branin(x)

x =

  -3.141545251350651e+00
   1.227500114531988e+01

ans =    3.978873817572968e-01


In [134]:
[W,x,iter] = metodoblhibrido(@easom,[4.5,4.5]','max',false);
x
easom(x)

x =

   3.141591902310620
   3.141591902310620

ans =   -9.999999999983067e-01


In [137]:
[W,x,iter] = metodoblhibrido(@shubert,[3,1]','max',false);
x
shubert(x)

x =

  -25.93306233708681
   36.27384942464656

ans = -186.7308669891247
